### 라이브러리 불러오기
필요한 라이브러리를 불러옵니다.  
설치 방법: https://github.com/ourownstory/neural_prophet

In [29]:
import pandas as pd
import numpy as np
from datetime import datetime
from neuralprophet import NeuralProphet

### 시간 변환
데이터 시간을 1 ~ 24 에서 0 ~ 23 로 변경합니다.

In [31]:
def convert_time(x):
    Ymd, HMS = x.split(' ')
    H, M, S = HMS.split(':')
    H = str(int(H)-1)
    HMS = ':'.join([H, M, S])
    return ' '.join([Ymd, HMS])

### 데이터 불러오기

In [33]:
# 데이터 불러오기
train_data = pd.read_csv('A:\study\data\ewp\energy.csv')
# 시간 변환
train_data['time'] = train_data['time'].apply(lambda x:convert_time(x))

### 훈련 데이터 만들기
ds = 시간, y = 데이터

In [35]:
column = 'dangjin_floating'
df = pd.DataFrame()
df['ds'] = train_data['time']
df['y'] = train_data[column]

### 훈련 및 예측

In [37]:
# 모델 설정
model = NeuralProphet()
# 훈련
loss = model.fit(df, freq="H")
# 예측용 데이터 프레임 만들기
df_pred = model.make_future_dataframe(df, periods=18000)
# 예측
predict = model.predict(df_pred)

INFO - (NP.forecaster._handle_missing_data) - dropped 24 NAN row in 'y'
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 64
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 61
 90%|█████████ | 289/320 [00:03<00:00, 92.37it/s]
INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 9.86E-02, min: 5.90E-01
INFO - (NP.utils_torch.lr_range_test) - learning rate range test selected lr: 3.25E-01
Epoch[61/61]: 100%|██████████| 61/61 [00:40<00:00,  1.50it/s, SmoothL1Loss=0.0161, MAE=67.7, MSE=1.04e+4, RegLoss=0]


### 제출 파일 생성

In [39]:
submission = pd.read_csv('A:\study\data\ewp\sample_submission.csv')

In [40]:
# 2021-02-01 ~ 2021-03-01
predict_1 = predict.copy()
predict_1 = predict_1.query('ds >= "2021-02-01 00:00:00"')
predict_1 = predict_1.query('ds < "2021-03-01 00:00:00"')

# 2021-06-09 ~ 2021-07-09
predict_2 = predict.copy()
predict_2 = predict_2.query('ds >= "2021-06-09 00:00:00"')
predict_2 = predict_2.query('ds < "2021-07-09 00:00:00"')

# 제출 파일 업데이트
submission[column] = list(predict_1['yhat1']) + list(predict_2['yhat1'])

In [41]:
submission.head()

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
0,2021-02-01 01:00:00,-29.988970,0,0,0
1,2021-02-01 02:00:00,-28.068808,0,0,0
2,2021-02-01 03:00:00,-27.118706,0,0,0
3,2021-02-01 04:00:00,-28.068150,0,0,0
4,2021-02-01 05:00:00,-28.209770,0,0,0
